# Imports

In [10]:
import pandas as pd
import numpy as np
from datetime import datetime
from functools import reduce
from tqdm.notebook import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import coo_matrix, hstack

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

%matplotlib inline

from sklearn import metrics
import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


from surprise import KNNWithMeans, KNNBasic
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

import pandas as pd

# Data downloads

In [4]:
links = pd.read_csv('/Users/user/Desktop/netology/rs/ml-latest-small/links.csv')
movies = pd.read_csv('/Users/user/Desktop/netology/rs/ml-latest-small/movies.csv')
ratings = pd.read_csv('/Users/user/Desktop/netology/rs/ml-latest-small/ratings.csv')
tags = pd.read_csv('/Users/user/Desktop/netology/rs/ml-latest-small/tags.csv')

# Data transforming

In [6]:
movies_with_tags = movies.join(tags.set_index('movieId'), on='movieId')

In [7]:
movies_with_tags_ratings = pd.merge(movies_with_tags, ratings,  how='left', 
                                    left_on=['movieId','userId'], right_on = ['movieId','userId'])

In [8]:
movies_with_tags_ratings.drop(['timestamp_x','timestamp_y'], axis = 1, inplace = True)
movies_with_tags_ratings.dropna(inplace=True)
movies_with_tags_ratings

,movieId,title,genres,userId,tag,rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,4.0
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,474.0,pixar,4.0
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,567.0,fun,3.5
3,2,Jumanji (1995),Adventure|Children|Fantasy,62.0,fantasy,4.0
4,2,Jumanji (1995),Adventure|Children|Fantasy,62.0,magic board game,4.0
...,...,...,...,...,...,...
11818,187595,Solo: A Star Wars Story (2018),Action|Adventure|Children|Sci-Fi,62.0,star wars,4.0
11840,193565,Gintama: The Movie (2010),Action|Animation|Comedy|Sci-Fi,184.0,anime,3.5
11841,193565,Gintama: The Movie (2010),Action|Animation|Comedy|Sci-Fi,184.0,comedy,3.5
11842,193565,Gintama: The Movie (2010),Action|Animation|Comedy|Sci-Fi,184.0,gintama,3.5


In [18]:
dataset = pd.DataFrame({
    'uid': movies_with_tags_ratings.userId,
    'iid': movies_with_tags_ratings.title,
    'rating': movies_with_tags_ratings.rating
})
dataset.head()

,uid,iid,rating
0,336.0,Toy Story (1995),4.0
1,474.0,Toy Story (1995),4.0
2,567.0,Toy Story (1995),3.5
3,62.0,Jumanji (1995),4.0
4,62.0,Jumanji (1995),4.0


In [29]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)

In [40]:
train, test = train_test_split(data, test_size=.15, random_state=10)

In [41]:
algo = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': True})
algo.fit(train)
test_pred = algo.test(test)
accuracy.rmse(test_pred, verbose=True)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.5979


0.5979403211824009

In [ ]:
scores = cross_val_score(
    clf, X, y, cv=5, scoring='f1_macro')
scores